In [3]:
import time
import numpy as np
from scipy.constants import elementary_charge, proton_mass 
from datetime import date
from datetime import datetime
import os
import vis_canonical_fluxtubes as vis

In [4]:
elementary_charge

1.6021766208e-19

In [5]:
import visit
visit.Launch()

True

In [4]:
database_prefix = '/Users/vonderlinden2/rsx_analysis/writing_to_vtk/output/2016-08-10-10-27'

In [5]:
proton_mass/elementary_charge

1.0439684840519176e-08

In [6]:
proton_mass

1.672621898e-27

In [7]:
elementary_charge

1.6021766208e-19

In [8]:
tan = (209, 178, 111, 255)
olive = (110, 117, 14, 255)

In [9]:
print "test %1.2e" % 12e5

test 1.20e+06


In [10]:
def define_expressions(alpha=8.1e5):
    visit.DefineVectorExpression("B", "{B_x, B_y, B_z}")
    visit.DefineVectorExpression("B_norm", "{B_norm_x, B_norm_y, B_norm_z}")
    visit.DefineVectorExpression("B_perp", "{B_x, B_y, 0}")
    visit.DefineVectorExpression("B_para", "{0, 0, B_z}")
    visit.DefineScalarExpression("B_para_scalar", "B_z")
    
    visit.DefineVectorExpression("J", "{j_x, j_y, j_z}")
    visit.DefineScalarExpression("J_mag", "sqrt(j_x^2 + j_y^2 + j_z^2)")
    visit.DefineVectorExpression("J_perp", "{j_x, j_y, 0}")
    visit.DefineVectorExpression("J_para", "{0, 0, j_z}")
    visit.DefineScalarExpression("J_para_scalar", "j_z")
    
    visit.DefineVectorExpression("u_e_norm", "{u_e_norm_x, u_e_norm_y, u_e_norm_z}")
    visit.DefineVectorExpression("u_e", "%1.2e * u_e_norm" % alpha)
    visit.DefineVectorExpression("u_e_perp", "{dot(u_e, {1, 0, 0}), dot(u_e, {0, 1, 0}), 0}")
    visit.DefineScalarExpression("u_e_para_scalar", "dot(u_e, {0, 0, 1})")
    
    
    visit.DefineVectorExpression("u_i_term1", "{u_i_term1_x, u_i_term1_y, u_i_term1_z}")
    visit.DefineVectorExpression("u_i_term1_perp", "{u_i_term1_x, u_i_term1_y, 0}")
    visit.DefineVectorExpression("u_i_term1_para", "{0, 0, u_i_term1_z}") 
    visit.DefineScalarExpression("u_i_term1_para_scalar", "u_i_term1_z") 
    
    visit.DefineVectorExpression("omega_i_term1", "{w_i_term1_x, w_i_term1_y, w_i_term1_z}")
    visit.DefineVectorExpression("omega_i_term1_perp", "{w_i_term1_x, w_i_term1_y, 0}")
    visit.DefineVectorExpression("omega_i_term1_para", "{0, 0, w_i_term1_z}")
    visit.DefineScalarExpression("omega_i_term1_para_scalar", "w_i_term1_z")
    
    visit.DefineVectorExpression("omega_i_term2", "{w_i_term2_x, w_i_term2_y, w_i_term2_z} * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_term2_perp", "{w_i_term2_x, w_i_term2_y, 0} * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_term2_para", "{0, 0, w_i_term2_z} * %1.2e" % alpha)
    visit.DefineScalarExpression("omega_i_term2_para_scalar", "w_i_term2_z * %1.2e" % alpha)
    
    visit.DefineVectorExpression("u_i", "u_i_term1 + u_e")
    visit.DefineVectorExpression("u_i_perp", "{dot(u_i, {1, 0, 0}), dot(u_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("u_i_para", "{0, 0, dot(u_i, {0, 0, 1})}")
    visit.DefineScalarExpression("u_i_para_scalar", "dot(u_i, {0, 0, 1})")
    
    visit.DefineVectorExpression("omega_i", "omega_i_term1 + omega_i_term2 * %1.2e" % alpha)
    visit.DefineVectorExpression("omega_i_perp", "{dot(omega_i, {1, 0, 0}), dot(omega_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("omega_i_para", "{0, 0, dot(omega_i, {0, 0, 1})}")
    visit.DefineScalarExpression("omega_i_para_scalar", "dot(omega_i, {0, 0, 1})")
    
    visit.DefineVectorExpression("Omega_i", "B +" + str(proton_mass) + "/" + str(elementary_charge) + "*omega_i")
    visit.DefineVectorExpression("Omega_i_perp", "{dot(Omega_i, {1, 0, 0}), dot(Omega_i, {0, 1, 0}), 0}")
    visit.DefineVectorExpression("Omega_i_para", "{0, 0, dot(Omega_i, {0, 0, 1})}")
    visit.DefineScalarExpression("Omega_i_para_scalar", "dot(Omega_i, {0, 0, 1})")

In [11]:
define_expressions(alpha=6e3)

In [12]:
def launch_points(center, plane=0.249, num_inner=80, num_outer=60):
    thetas = np.linspace(0, 3./4.*np.pi, num_outer)
    thetas = np.concatenate((thetas, np.linspace(5./4.*np.pi, 2.*np.pi, num_outer)))
    x_outer = 0.01 * np.cos(thetas) + center[0]
    y_outer = 0.01 * np.sin(thetas) + center[1]
    z_outer = plane * np.ones((x_outer.size))
    points_outer = np.empty((x_outer.size + y_outer.size + z_outer.size))
    points_outer[0::3] = x_outer
    points_outer[1::3] = y_outer
    points_outer[2::3] = z_outer
    points_outer = tuple(points_outer)

    thetas = np.linspace(0, 2.*np.pi, num_inner)
    x_inner = 0.005 * np.cos(thetas) + center[0]
    y_inner = 0.005 * np.sin(thetas) + center[1]
    z_inner = plane * np.ones(x_inner.size)
    points_inner = np.empty((x_inner.size + y_inner.size + z_inner.size))
    points_inner[0::3] = x_inner
    points_inner[1::3] = y_inner 
    points_inner[2::3] = z_inner
    points_inner = tuple(points_inner)
    
    return points_outer, points_inner

In [13]:
def setup_current_pseudocolor(colortable="Greens"):
    r"""
    """
    visit.AddPlot("Pseudocolor", "J_mag", 1, 0)
    PseudocolorAtts = visit.PseudocolorAttributes()
    PseudocolorAtts.scaling = PseudocolorAtts.Linear  
    PseudocolorAtts.limitsMode = PseudocolorAtts.OriginalData 
    PseudocolorAtts.colorTableName = colortable
    visit.SetPlotOptions(PseudocolorAtts)
    visit.AddOperator("Slice", 0)
    SliceAtts = visit.SliceAttributes()
    SliceAtts.originType = SliceAtts.Intercept 
    SliceAtts.originIntercept = 0.249
    SliceAtts.axisType = SliceAtts.ZAxis
    SliceAtts.project2d = 0
    visit.SetOperatorOptions(SliceAtts, 0)
    return PseudocolorAtts, SliceAtts

In [14]:
def setup_massless_electron_canonical_flux_tubes(points_outer, points_inner):
    r"""
    """
    visit.AddPlot("Streamline", "B", 1, 0)
    StreamlineAtts_outer = visit.StreamlineAttributes()
    StreamlineAtts_outer.sourceType = StreamlineAtts_outer.SpecifiedPointList
    StreamlineAtts_outer.SetPointList(points_outer)
    StreamlineAtts_outer.coloringMethod = StreamlineAtts_outer.Solid  
    StreamlineAtts_outer.colorTableName = "Default"
    StreamlineAtts_outer.singleColor = (255, 0, 0, 255)
    visit.SetPlotOptions(StreamlineAtts_outer)

    visit.AddPlot("Streamline", "B", 1, 0)
    StreamlineAtts_inner = visit.StreamlineAttributes()
    StreamlineAtts_inner.sourceType = StreamlineAtts_inner.SpecifiedPointList
    StreamlineAtts_inner.SetPointList(points_inner)
    StreamlineAtts_inner.coloringMethod = StreamlineAtts_inner.Solid
    StreamlineAtts_inner.colorTableName = "Default"
    StreamlineAtts_inner.singleColor = (190, 64, 0, 255)
    visit.SetPlotOptions(StreamlineAtts_inner)
    
    return StreamlineAtts_outer, StreamlineAtts_inner

In [15]:
def setup_inner_outer_ion_canonical_flux_tubes(points_outer, points_inner,
                                   outer_color=tan, inner_color=olive):
    r"""
    """
    visit.AddPlot("Streamline", "Omega_i", 1, 0)
    StreamlineAtts_outer = visit.StreamlineAttributes()
    StreamlineAtts_outer.sourceType = StreamlineAtts_outer.SpecifiedPointList
    StreamlineAtts_outer.SetPointList(points_outer)
    StreamlineAtts_outer.coloringMethod = StreamlineAtts_outer.Solid  
    StreamlineAtts_outer.colorTableName = "Default"
    StreamlineAtts_outer.singleColor = outer_color
    visit.SetPlotOptions(StreamlineAtts_outer)

    visit.AddPlot("Streamline", "Omega_i", 1, 0)
    StreamlineAtts_inner = visit.StreamlineAttributes()
    StreamlineAtts_inner.sourceType = StreamlineAtts_inner.SpecifiedPointList
    StreamlineAtts_inner.SetPointList(points_inner)
    StreamlineAtts_inner.coloringMethod = StreamlineAtts_inner.Solid
    StreamlineAtts_inner.colorTableName = "Default"
    StreamlineAtts_inner.singleColor = inner_color
    visit.SetPlotOptions(StreamlineAtts_inner)
    
    return StreamlineAtts_outer, StreamlineAtts_inner

In [16]:
def setup_forward_backward_ion_canonical_flux_tubes(points_foward, points_backward,
                                                    forward_color=tan, backward_color=olive):
    r"""
    """
    visit.AddPlot("Streamline", "Omega_i", 1, 0)
    
    StreamlineAtts_forward = visit.StreamlineAttributes()
    StreamlineAtts_forward.sourceType = StreamlineAtts_forward.SpecifiedPointList
    StreamlineAtts_forward.SetPointList(points_foward)
    StreamlineAtts_forward.coloringMethod = StreamlineAtts_forward.Solid  
    StreamlineAtts_forward.colorTableName = "Default"
    StreamlineAtts_forward.singleColor = forward_color
    StreamlineAtts_forward.integrationDirection = StreamlineAtts_forward.Forward
    visit.SetPlotOptions(StreamlineAtts_forward)

    visit.AddPlot("Streamline", "Omega_i", 1, 0)
    StreamlineAtts_backward = visit.StreamlineAttributes()
    StreamlineAtts_backward.sourceType = StreamlineAtts_backward.SpecifiedPointList
    StreamlineAtts_backward.SetPointList(points_backward)
    StreamlineAtts_backward.coloringMethod = StreamlineAtts_backward.Solid
    StreamlineAtts_backward.colorTableName = "Default"
    StreamlineAtts_backward.singleColor = backward_color
    StreamlineAtts_backward.integrationDirection = StreamlineAtts_backward.Backward
    visit.SetPlotOptions(StreamlineAtts_backward)
    
    return StreamlineAtts_forward, StreamlineAtts_backward

In [17]:
today = str(date.today())
out_dir = '../output/' + today
try:
    os.makedirs(out_dir)
except:
    pass

visit.OpenDatabase(database_prefix + '/Bdot_triple_probe_quantities*.vtk database')
field_nulls = np.loadtxt('../../centroid_fitting/output/2016-08-06/field_nulls_0249.txt')
field_nulls_357 = np.loadtxt('../../centroid_fitting/output/2016-08-06/field_nulls_0357.txt')
electron_points_outer, electron_points_inner = launch_points(field_nulls[0], num_inner=10)
ion_points_outer, ion_points_inner = launch_points(field_nulls[0], plane=0.249, num_inner=80)
define_expressions()
setup_current_pseudocolor()
(electron_StreamlineAtts_outer, 
 electron_StreamlineAtts_inner) = setup_massless_electron_canonical_flux_tubes(electron_points_inner, 
                                                                               electron_points_inner)
(ion_StreamlineAtts_forward, 
 ion_StreamlineAtts_backward) = setup_forward_backward_ion_canonical_flux_tubes(ion_points_inner, 
                                                                                ion_points_inner)


In [18]:
view = visit.GetView3D()
view.SetViewNormal((-0.731293, 0.40847, 0.546227))
view.SetFocus((0.00202222, 0.000976744, 0.331997))
view.SetViewUp((0.322268, 0.91274, -0.251095))
view.SetViewAngle(30)
view.SetParallelScale(0.088383)
view.SetNearPlane(-0.176766)
view.SetImagePan((0, 0))
view.SetImageZoom(1)
view.SetPerspective(1)
view.SetEyeAngle(2)
view.SetCenterOfRotationSet(0)
view.SetCenterOfRotation((0.00202222, 0.000976744, 0.331997))
view.SetAxis3DScaleFlag(0)
view.SetAxis3DScales((1,1,1))
view.SetShear((0,0,1))
view.SetWindowValid(0)
visit.SetView3D(view)

1L

In [19]:
visit.DrawPlots()

1L

In [ ]:
output_path = out_dir + '/ion_canonical_flux_tube'
ending = '.png'
plane = 0.249
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)

for time_point in (range(0, 14) + range(17, 20)):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = launch_points(field_nulls_357[time_point], plane)
    e_points_outer, e_points_inner = launch_points(field_nulls_357[time_point], plane, num_inner=10)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(e_points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetActivePlots(3)
    ion_StreamlineAtts_forward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_forward)
    
    visit.SetActivePlots(4)
    ion_StreamlineAtts_backward.SetPointList(points_inner)
    visit.SetPlotOptions(ion_StreamlineAtts_backward)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

In [22]:
!ffmpeg -framerate 1/2 -i ../output/2016-08-18/ion_canonical_flux_tube%04d.png -r 1/2 -pix_fmt yuv420p \
 -vcodec libx264 -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -y ../output/2016-08-18/out.mp4

ffmpeg version 2.8.1 Copyright (c) 2000-2015 the FFmpeg developers
  built with Apple LLVM version 6.0 (clang-600.0.56) (based on LLVM 3.5svn)
  configuration: --prefix=/Users/vonderlinden2/homebrew/Cellar/ffmpeg/2.8.1_1 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libvo-aacenc --enable-libxvid --enable-vda
  libavutil      54. 31.100 / 54. 31.100
  libavcodec     56. 60.100 / 56. 60.100
  libavformat    56. 40.101 / 56. 40.101
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 40.101 /  5. 40.101
  libavresample   2.  1.  0 /  2.  1.  0
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.101 /  1.  2.101
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from '../output/2016-08-18/ion_canonical_flux_tube%04d.png':
  Duration: 00:00:42.00, start: 0.000000, bitrate: N/A
    Stream #0:0: V